# First Order Markov Process Simulation (S1-S3)

In [ ]:
import numpy as np
np.random.seed(0)
from gendata import genbasicseqs, plotprobs, genfirstorder
from ctw import CTW
import pickle

N=300000
randparams=True
num_trials=100
thresh=0.001
ks = [1,2,3]

numx = 4
numy = 4

def kl(d1,d2):
    kl = 0
    for i in range(len(d1)):
        kl += d1[i]*np.log2(d1[i]/d2[i])
    return kl

def avg_kl(px1,px2):
    if px1.shape != px2.shape:
        raise Exception
    N = px1.shape[1]
    tot = 0
    for n in range(N):
        tot += kl(px1[:,n],px2[:,n])
    return tot/N

def gen_params(numx,numy,case=1):
    xparams = [[None for y in range(numy)] for x in range(numx)] 
    yparams = [[None for y in range(numy)] for x in range(numx)] 
    if case == 1: 
        for y in range(numy):
            xparam = np.random.exponential(size=numx)
            xparam = xparam/sum(xparam)
            yparam = np.random.exponential(size=numy)
            yparam = yparam/sum(yparam)
            for x in range(numx):
                xparams[x][y] = xparam
                yparams[x][y] = yparam
    
    elif case == 2:
        for y in range(numy):
            yparam = np.random.exponential(size=numy)
            yparam = yparam/sum(yparam)
            for x in range(numx):
                xparam = np.random.exponential(size=numx)
                xparam = xparam/sum(xparam)
                xparams[x][y] = xparam
                yparams[x][y] = yparam
        
    elif case == 2:
        for y in range(numy):
            xparam = np.random.exponential(size=numx)
            xparam = xparam/sum(xparam)
            for x in range(numx):
                yparam = np.random.exponential(size=numy)
                yparam = yparam/sum(yparam)
                xparams[x][y] = xparam
                yparams[x][y] = yparam
                
    else:
        for y in range(numy):
            for x in range(numx):
                xparam = np.random.exponential(size=numx)
                xparam = xparam/sum(xparam)
                yparam = np.random.exponential(size=numy)
                yparam = yparam/sum(yparam)
                xparams[x][y] = xparam
                yparams[x][y] = yparam
                

                
    return [np.asarray(xparams),np.asarray(yparams)]

## (S1): $Y\rightarrow Y$, $Y\rightarrow X$, $X\not\rightarrow X$, $X\not\rightarrow Y$ 

In [ ]:
TDIs = np.zeros((num_trials,len(ks)))
PDIs = np.zeros((num_trials,len(ks)))
for trial in range(num_trials):
    print('--------------- Trial %i ---------------'%(trial+1))
    
    if randparams:
        params = gen_params(numx,numy,case=1)

    else:
        px1_y0 = 0.9                  #p(x_t=1|y_{t-1}=0) 
        px_y0 = [px1_y0,1-px1_y0]     #p(x|y=0)
        px1_y1 = 0.1                  #p(x=1|y=1)  
        px_y1 = [px1_y1,1-px1_y1]     #p(x|y=1)
        py1_y0 = 0.1                  #p(y=1|y=0)  
        py_y0 = [py1_y0,1-py1_y0]     #p(y|y=0)
        py1_y1 = 0.9                  #p(y=1|y=1)  
        py_y1 = [py1_y1,1-py1_y1]     #p(y|y=1)

        xparams = np.asarray([[px_y0,px_y1],[px_y0,px_y1]])
        yparams = np.asarray([[py_y0,py_y1],[py_y0,py_y1]])

        params = [xparams,yparams]

    x,y = genfirstorder(N=N,params=params,plot=False,plot_samples=100,numx=numx)

    ctw_full = CTW(depth=1,symbols=numx,sidesymbols=numy)
    px_full = ctw_full.predict_sequence(x,y)
    for k in ks:
        print('k=%i'%k)
        ctw_marg = CTW(depth=k,symbols=numx)
        px_marg = ctw_marg.predict_sequence(x)
        DIk = avg_kl(px_full[:,k-1:],px_marg)
        print('EDI = %1.3f'%DIk)
        ctw_part = CTW(depth=k+1,symbols=numx,sidesymbols=numy,staleness=k)
        px_part = ctw_part.predict_sequence(x,y)
        PDIk = avg_kl(px_full[:,k:],px_part)
        print('PDI = %1.3f'%PDIk)
        TDIs[trial,k-1]=DIk
        PDIs[trial,k-1]=PDIk
        if(DIk-PDIk < thresh):
            print('Converged!')
            break
            
pickle.dump((TDIs,PDIs), open('case1n%i.pickle'%num_trials, 'wb'))

## (S2): $Y\rightarrow Y$, $Y\rightarrow X$, $X\rightarrow X$, $X\not\rightarrow Y$ 

In [ ]:
TDIs = np.zeros((num_trials,len(ks)))
PDIs = np.zeros((num_trials,len(ks)))
for trial in range(num_trials):
    print('--------------- Trial %i ---------------'%(trial+1))
    
    if randparams:
        params = gen_params(numx,numy,case=2)

    else:
        px1_x0y0 = 0.9               
        px_x0y0 = [px1_x0y0,1-px1_x0y0]  
        px1_x0y1 = 0.1               
        px_x0y1 = [px1_x0y1,1-px1_x0y1]  
        px1_x1y0 = 0.1               
        px_x1y0 = [px1_x1y0,1-px1_x1y0]  
        px1_x1y1 = 0.9               
        px_x1y1 = [px1_x1y1,1-px1_x1y1]  
        py1_y0 = 0.4               
        py_y0 = [py1_y0,1-py1_y0]
        py1_y1 = 0.9               
        py_y1 = [py1_y1,1-py1_y1]  

        xparams = np.asarray([[px_x0y0,px_x0y1],[px_x1y0,px_x1y1]])
        yparams = np.asarray([[py_y0,py_y1],[py_y0,py_y1]])

        params = [xparams,yparams]

    x,y = genfirstorder(N=N,params=params,plot=False,plot_samples=100,numx=numx)

    ctw_full = CTW(depth=1,symbols=numx,sidesymbols=numy)
    px_full = ctw_full.predict_sequence(x,y)
    for k in ks:
        print('k=%i'%k)
        ctw_marg = CTW(depth=k,symbols=numx)
        px_marg = ctw_marg.predict_sequence(x)
        DIk = avg_kl(px_full[:,k-1:],px_marg)
        print('EDI = %1.3f'%DIk)
        ctw_part = CTW(depth=k+1,symbols=numx,sidesymbols=numy,staleness=k)
        px_part = ctw_part.predict_sequence(x,y)
        PDIk = avg_kl(px_full[:,k:],px_part)
        print('PDI = %1.3f'%PDIk)
        TDIs[trial,k-1]=DIk
        PDIs[trial,k-1]=PDIk        
        if(DIk-PDIk < thresh):
            print('Converged!')
            break
            
pickle.dump((TDIs,PDIs), open('case2n%i.pickle'%num_trials, 'wb'))

## (S3): $Y\rightarrow Y$, $Y\rightarrow X$, $X\rightarrow X$, $X\rightarrow Y$ 

In [ ]:
TDIs = np.zeros((num_trials,len(ks)))
PDIs = np.zeros((num_trials,len(ks)))
for trial in range(num_trials):
    print('--------------- Trial %i ---------------'%(trial+1))

    if randparams:
        params = gen_params(numx,numy,case=4)

    else:
        px1_x0y0 = 0.9               
        px_x0y0 = [px1_x0y0,1-px1_x0y0]  
        px1_x0y1 = 0.1               
        px_x0y1 = [px1_x0y1,1-px1_x0y1]  
        px1_x1y0 = 0.1               
        px_x1y0 = [px1_x1y0,1-px1_x1y0]  
        px1_x1y1 = 0.9               
        px_x1y1 = [px1_x1y1,1-px1_x1y1]  
        py1_x0y0 = 0.8               
        py_x0y0 = [py1_x0y0,1-py1_x0y0]
        py1_x0y1 = 0.9               
        py_x0y1 = [py1_x0y1,1-py1_x0y1]
        py1_x1y0 = 0.1               
        py_x1y0 = [py1_x1y0,1-py1_x1y0]
        py1_x1y1 = 0.2               
        py_x1y1 = [py1_x1y1,1-py1_x1y1]  

        xparams = np.asarray([[px_x0y0,px_x0y1],[px_x1y0,px_x1y1]])
        yparams = np.asarray([[py_x0y0,py_x0y1],[py_x1y0,py_x1y1]])

        params = [xparams,yparams]

    x,y = genfirstorder(N=N,params=params,plot=False,plot_samples=100,numx=numx)

    ctw_full = CTW(depth=1,symbols=numx,sidesymbols=numy)
    px_full = ctw_full.predict_sequence(x,y)
    for k in ks:
        print('k=%i'%k)
        ctw_marg = CTW(depth=k,symbols=numx)
        px_marg = ctw_marg.predict_sequence(x)
        DIk = avg_kl(px_full[:,k-1:],px_marg)
        print('EDI = %1.3f'%DIk)
        ctw_part = CTW(depth=k+1,symbols=numx,sidesymbols=numy,staleness=k)
        px_part = ctw_part.predict_sequence(x,y)
        PDIk = avg_kl(px_full[:,k:],px_part)
        print('PDI = %1.3f'%PDIk)
        TDIs[trial,k-1]=DIk
        PDIs[trial,k-1]=PDIk
        if(DIk-PDIk < thresh):
            print('Converged!')
            break
            
pickle.dump((TDIs,PDIs), open('case3n%i.pickle'%num_trials, 'wb'))